# Image tiling for annotation

#### Meanings of arguments
- ```-ratioheight``` : proportion of tile  w.r.t height of image. Example 0.5 means dividing the image in two bands w.r.t height.
- ```-ratiowidth``` : proportion of tile w.r.t to width of image. Example 1.0 means the width of the tile is the same as the image.
- ```-overlapfactor``` : percentage of overlap. It should be less than 1.
- ```-rmheight``` : percentage of height to remove or crop at bottom and top
- ```-rmwidth``` : percentage of width to remove or crop on each side of the image
- ```-pattern``` : "**/*.JPG" will get all .JPG images in directory and subdirectories. On windows it will get both .JPG and .jpg. On unix it will only get .JPG images


In [ ]:
# New script for tiling data
# images_to_tile = r"D:\PhD\Data per camp\Extra training data\savmap_dataset_v2\raw_data\images"
# destination_directory = r"D:\PhD\Data per camp\Extra training data\savmap_dataset_v2\raw_data\images-tiled"
!python ../HerdNet/tools/patcher.py "D:\PhD\Data per camp\Dry season\Kapiri\Camp 6-8\Rep 2" 0 0 0 -overlapfactor 0.1  -ratiowidth 0.33334 -ratioheight 0.5 -rmheight 0.21 -rmwidth 0.1 -dest "D:\PhD\Data per camp\Dry season\Kapiri\Camp 6-8\Rep 2 - tiled" -pattern "**/*.JPG"

# Pre-annotating data for Labelstudio

In [1]:
from datalabeling.annotator import Annotator
import os
from pathlib import Path
import torch

### Creating a JSON file to be uuploaded to Label studio

In [ ]:
# Example
# provide correct alias, "pt", "onnx"
alias = "last" # the aliases are found in mlflow tracker UI
name = "obb-detector" # detector, "obb-detector"
handler = Annotator(mlflow_model_alias=alias,
                    mlflow_model_name=name,
                    is_yolo_obb=True,
                    # dotenv_path="../.env"
                    )
path_img_dir=r"D:\PhD\Data per camp\Dry season\Kapiri\Camp 6-8\Rep 2 - tiled"
root="D:\\"
save_json_path = os.path.join(Path(path_img_dir).parent,
                              f"{Path(path_img_dir).name}_preannotation_label-studio.json")

# build and saves json
directory_preds = handler.build_upload_json(path_img_dir=path_img_dir,
                                            root=root,
                                            save_json_path=save_json_path,
                                            pattern="**/*.JPG")

### Pre-annotating an existing project using Label studio API
It seems that it will not work well (i.e. filtering) with older projects created prior to Label studio software update.
It is the **recommended way of pre-annotating data in Labelstudio**.

In [2]:
from dotenv import load_dotenv
load_dotenv('../.env')

True

In [ ]:
# provide correct alias, "pt", "onnx"
alias = "last"
name = "obb-detector" # detector, "obb-detector"
handler = Annotator(mlflow_model_alias=alias,
                    mlflow_model_name=name,
                    confidence_threshold=0.1,
                    dotenv_path="../.env")
project_id = ... # insert correct project_id by loooking at the url
handler.upload_predictions(project_id=project_id)

To speed up inference on intel, make changes inn ultralytics/nn/autobackend.py:
```
- device_name = "AUTO:NPU,GPU,CPU" # CPU, GPU, NPU, AUTO,"AUTO:GPU,NPU"
- inference_mode = "LATENCY" # OpenVINO inference modes are 'LATENCY', 'THROUGHPUT' (not recommended), or 'CUMULATIVE_THROUGHPUT'
- LOGGER.info(f"Using OpenVINO {inference_mode} mode for inference...")
- ov_compiled_model = core.compile_model(
                ov_model,
                device_name=device_name,  # AUTO selects best available device, do not modify
                config={"PERFORMANCE_HINT": inference_mode,
                        "CACHE_DIR": os.environ["OPENVINO_CACHE_MODEL"]}, # make sure to set environment variable
            )
```

In [3]:
# using path_to_weights
# go to ultralytics.nn.autobackend to modify ov_compiled device to "AUTO:NPU,GPU,CPU"
handler = Annotator(path_to_weights=r"C:\Users\FADELCO\OneDrive\Bureau\datalabeling\models\best_openvino_model",
                    is_yolo_obb=True,
                    tilesize=1280,
                    overlapratio=0.1,
                    confidence_threshold=0.2,
                    device="cpu", # "cpu", "cuda"
                    dotenv_path="../.env")
project_id = 1 # insert correct project_id by loooking at the url
handler.upload_predictions(project_id=project_id,top_n=0)

11/16/2024 10:57:53 - INFO - datalabeling.annotator.models -   Computing device: cpu


Loading C:\Users\FADELCO\OneDrive\Bureau\datalabeling\models\best_openvino_model for OpenVINO inference...
Available openvino devices are: ['CPU', 'GPU', 'NPU']. Using device=AUTO:NPU,GPU,CPU.
Using OpenVINO LATENCY mode for inference...


Uploading predictions:   0%|          | 0/654 [00:00<?, ?it/s]

Performing prediction on 12 slices.


Uploading predictions:   0%|          | 1/654 [00:04<43:00,  3.95s/it]

...

Performing prediction on 12 slices.


Uploading predictions:  15%|█▍        | 95/654 [04:55<28:40,  3.08s/it]

Performing prediction on 12 slices.


Uploading predictions:  15%|█▍        | 95/654 [04:57<29:12,  3.14s/it]


KeyboardInterrupt: 

In [9]:
from label_studio_ml.utils import get_local_path
from urllib.parse import unquote, quote
import os

In [ ]:
path = unquote("/data/local-files/?d=savmap_dataset_v2%5Cimages%5C003a34ee6b7841e6851b8fe511ebe102.JPG")
get_local_path(path)

11/16/2024 10:55:45 - WARNING - label_studio_tools.core.utils.io -   Using `localhost` (http://localhost:8080) in LABEL_STUDIO_URL, `localhost` is not accessible inside of docker containers. You can check your IP with utilities like `ifconfig` and set it as LABEL_STUDIO_URL.


'D:\\savmap_dataset_v2\\images\\003a34ee6b7841e6851b8fe511ebe102.JPG'

# Inference with Sahi

In [1]:
from ultralytics import YOLO
from sahi.predict import get_sliced_prediction
from PIL import Image
import time
import numpy as np
from datalabeling.annotator import Detector
from dotenv import load_dotenv

In [2]:
# load env variable, loads model cache location!!
load_dotenv('../.env')

True

In [3]:
IMAGE_PATH = r"D:\savmap_dataset_v2\images_splits\00a033fefe644429a1e0fcffe88f8b39_1.JPG"

To speed up inference on intel, make changes inn ultralytics/nn/autobackend.py:
```
- device_name = "AUTO:NPU,GPU,CPU" # CPU, GPU, NPU, AUTO,"AUTO:GPU,NPU"
- inference_mode = "LATENCY" # OpenVINO inference modes are 'LATENCY', 'THROUGHPUT' (not recommended), or 'CUMULATIVE_THROUGHPUT'
- LOGGER.info(f"Using OpenVINO {inference_mode} mode for inference...")
- ov_compiled_model = core.compile_model(
                ov_model,
                device_name=device_name,  # AUTO selects best available device, do not modify
                config={"PERFORMANCE_HINT": inference_mode,
                        "CACHE_DIR": os.environ["OPENVINO_CACHE_MODEL"]}, # make sure to set environment variable
            )
```

In [4]:
# Define detector
# to speed up inference on intel, make
model = Detector(path_to_weights=r"C:\Users\FADELCO\OneDrive\Bureau\datalabeling\models\best_openvino_model",
                confidence_threshold=0.1,
                overlap_ratio=0.1,
                tilesize=1280,
                device='CPU',
                is_yolo_obb=True)

11/16/2024 10:06:04 - INFO - datalabeling.annotator.models -   Computing device: CPU


Loading C:\Users\FADELCO\OneDrive\Bureau\datalabeling\models\best_openvino_model for OpenVINO inference...
Available openvino devices are: ['CPU', 'GPU', 'NPU']. Using device=AUTO:NPU,GPU,CPU.
Using OpenVINO LATENCY mode for inference...


In [7]:
image = Image.open(IMAGE_PATH)

while True:
    start_time = time.perf_counter()
    model.predict(image)
    end_time = time.perf_counter()
    print(f"Device took {end_time-start_time:.2f} seconds.")

Performing prediction on 4 slices.
Device took 1.39 seconds.
Performing prediction on 4 slices.
Device took 1.24 seconds.
Performing prediction on 4 slices.
Device took 1.18 seconds.
Performing prediction on 4 slices.
Device took 1.10 seconds.
Performing prediction on 4 slices.
Device took 1.10 seconds.
Performing prediction on 4 slices.
Device took 1.17 seconds.
Performing prediction on 4 slices.
Device took 1.14 seconds.
Performing prediction on 4 slices.
Device took 1.19 seconds.
Performing prediction on 4 slices.
Device took 1.28 seconds.
Performing prediction on 4 slices.
Device took 1.30 seconds.
Performing prediction on 4 slices.
Device took 1.20 seconds.
Performing prediction on 4 slices.
Device took 1.31 seconds.
Performing prediction on 4 slices.
Device took 1.20 seconds.
Performing prediction on 4 slices.
Device took 1.21 seconds.
Performing prediction on 4 slices.
Device took 1.31 seconds.
Performing prediction on 4 slices.
Device took 1.18 seconds.
Performing prediction on

KeyboardInterrupt: 

In [9]:
# inference with openvino
import openvino as ov
import openvino.properties.hint as hints
import torch
import torchvision.transforms as F
from ultralytics.utils import DEFAULT_CFG
from ultralytics.cfg import get_cfg
from ultralytics.data.converter import coco80_to_coco91_class

# load validator
args = get_cfg(cfg=DEFAULT_CFG)
det_model = YOLO(r"C:\Users\FADELCO\OneDrive\Bureau\datalabeling\models\best.pt")
det_validator = det_model.task_map[det_model.task]["validator"](args=args)
det_validator.is_coco = True
det_validator.class_map = coco80_to_coco91_class()
det_validator.names = det_model.model.names
det_validator.metrics.names = det_validator.names
det_validator.nc = det_model.model.model[-1].nc
det_validator.stride = 32
args = get_cfg(cfg=DEFAULT_CFG)
det_model = YOLO(r"C:\Users\FADELCO\OneDrive\Bureau\datalabeling\models\best.pt")

core = ov.Core()
det_model_path = r"C:\Users\FADELCO\OneDrive\Bureau\datalabeling\models\best_openvino_model\best.xml"
det_ov_model = core.read_model(det_model_path)

device = "AUTO:NPU,GPU" # CPU, NPU, GPU "AUTO:NPU,GPU,CPU" 

# reshaping
input_layer = det_ov_model.input(0)
output_layer = det_ov_model.output(0)
new_shape = ov.PartialShape([2, 3, 1280, 1280])
det_ov_model.reshape({input_layer.any_name: new_shape})

ov_config = {hints.performance_mode: hints.PerformanceMode.THROUGHPUT,
             "CACHE_DIR": '../models/model_cache'}

if ("GPU" in core.available_devices) and device=="GPU":
    ov_config["GPU_DISABLE_WINOGRAD_CONVOLUTION"] = "YES"
det_compiled_model = core.compile_model(det_ov_model, device, ov_config)

def infer(image):
    image = det_validator.preprocess({"img":image,"batch_idx":torch.Tensor([0]),
                                      "cls":torch.Tensor([0]),
                                      "bboxes":torch.Tensor([0.,0.,0.,0.])})["img"]
    results = det_compiled_model(image)
    preds = torch.from_numpy(results[det_compiled_model.output(0)])
    return det_validator.postprocess(preds) #torch.from_numpy(result[0])

In [11]:
dummy_img = np.random.random(size=(2,3,1280,1280))
preds = det_compiled_model(dummy_img)

In [18]:
preds.values()

dict_values([array([[[     7.4219,      14.062,      20.094, ...,        1202,        1235,        1263],
        [     7.6016,      5.3047,      5.6641, ...,        1272,        1271,        1275],
        [     13.695,      19.766,       26.25, ...,      174.75,      178.38,      200.38],
        [     14.992,      13.727,      13.766, ...,         186,       187.5,      201.12],
        [ 0.00034332,  0.00015843,  0.00010151, ...,           0,           0,           0],
        [  -0.066345,   -0.036407,   -0.020706, ...,     -0.6333,    -0.64941,    -0.68408]],

       [[     8.1875,      14.195,      21.688, ...,        1202,        1235,        1263],
        [     5.7344,      5.4609,      5.7891, ...,        1272,        1271,        1275],
        [     14.961,      21.109,      28.531, ...,      176.25,         179,      200.75],
        [     13.141,      13.938,      15.023, ...,      186.75,      188.25,      201.12],
        [ 0.00013232,  9.9957e-05,           0, ...,   

In [4]:
image = Image.open(IMAGE_PATH)
image = F.PILToTensor()(image)[None,:,:1280,:1280]
# _ = infer(image)

In [5]:
# inference with pt
model = YOLO(r"C:\Users\FADELCO\OneDrive\Bureau\datalabeling\models\best.pt",task='obb')

In [8]:
model(image/255.)


0: 1280x1280 894.2ms
Speed: 0.0ms preprocess, 894.2ms inference, 0.0ms postprocess per image at shape (1, 3, 1280, 1280)


[ultralytics.engine.results.Results object with attributes:
 
 boxes: None
 keypoints: None
 masks: None
 names: {0: 'wildlife'}
 obb: ultralytics.engine.results.OBB object
 orig_img: array([[[130, 111,  79],
         [129, 110,  78],
         [128, 109,  77],
         ...,
         [157, 139, 103],
         [168, 150, 114],
         [173, 155, 119]],
 
        [[130, 111,  79],
         [128, 109,  77],
         [127, 108,  76],
         ...,
         [152, 134,  98],
         [158, 140, 104],
         [165, 147, 111]],
 
        [[129, 110,  78],
         [127, 108,  76],
         [125, 106,  74],
         ...,
         [147, 129,  93],
         [151, 133,  97],
         [163, 145, 109]],
 
        ...,
 
        [[147, 131,  95],
         [141, 125,  89],
         [136, 120,  84],
         ...,
         [139, 121,  85],
         [144, 125,  92],
         [146, 127,  94]],
 
        [[147, 129,  91],
         [142, 124,  86],
         [138, 120,  82],
         ...,
         [130, 112

In [9]:
# inference with openvino
model_vino = YOLO(r"C:\Users\FADELCO\OneDrive\Bureau\datalabeling\models\best_openvino_model",task='obb')

In [15]:
model_vino(image/255.)


0: 1280x1280 189.7ms
Speed: 0.0ms preprocess, 189.7ms inference, 17.0ms postprocess per image at shape (1, 3, 1280, 1280)


[ultralytics.engine.results.Results object with attributes:
 
 boxes: None
 keypoints: None
 masks: None
 names: {0: 'wildlife'}
 obb: ultralytics.engine.results.OBB object
 orig_img: array([[[130, 111,  79],
         [129, 110,  78],
         [128, 109,  77],
         ...,
         [157, 139, 103],
         [168, 150, 114],
         [173, 155, 119]],
 
        [[130, 111,  79],
         [128, 109,  77],
         [127, 108,  76],
         ...,
         [152, 134,  98],
         [158, 140, 104],
         [165, 147, 111]],
 
        [[129, 110,  78],
         [127, 108,  76],
         [125, 106,  74],
         ...,
         [147, 129,  93],
         [151, 133,  97],
         [163, 145, 109]],
 
        ...,
 
        [[147, 131,  95],
         [141, 125,  89],
         [136, 120,  84],
         ...,
         [139, 121,  85],
         [144, 125,  92],
         [146, 127,  94]],
 
        [[147, 129,  91],
         [142, 124,  86],
         [138, 120,  82],
         ...,
         [130, 112

In [16]:
core.available_devices

['CPU', 'GPU', 'NPU']

In [10]:
sahi_model_obb = Detector(path_to_weights=r"C:\Users\FADELCO\OneDrive\Bureau\datalabeling\models\best_openvino_model",
                    confidence_threshold=0.6,
                    overlap_ratio=0.1,
                    tilesize=640,
                    is_yolo_obb=True)

11/15/2024 13:35:56 - INFO - datalabeling.annotator.models -   Computing device: cpu


Loading C:\Users\FADELCO\OneDrive\Bureau\datalabeling\models\best_openvino_model for OpenVINO inference...
Using OpenVINO CUMULATIVE_THROUGHPUT mode for batch=16 inference...


In [11]:
image_path = r"D:\savmap_dataset_v2\images\0d1ba3c424ad4414ac37dbd0c93460ea.JPG"
image = Image.open(image_path)
print(image.size)

(4000, 3000)


In [12]:
result = sahi_model_obb.predict(image,False)

Performing prediction on 42 slices.


In [8]:
result

In [9]:
result.export_visuals('../.tmp')

In [ ]:
image_path = r"D:\PhD\Data per camp\Dry season\Kapiri\Camp 6-8\Rep 1 - tiled\DJI_20231003081043_0016_1.JPG"

In [ ]:
# plt.imshow(tile)
# plt.show()

# YOLO data_config.yaml 

In [ ]:
import yaml
import json
from arguments import Arguments

In [ ]:
# load yaml
with open(r"D:\PhD\Data per camp\IdentificationDataset\data_config.yaml",'r') as file:
    yolo_config = yaml.load(file,Loader=yaml.FullLoader)
yolo_config

In [ ]:
# load label mapping
args = Arguments()
with open(r"D:\PhD\Data per camp\IdentificationDataset\label_mapping.json",'r') as file:
    label_map = json.load(file)
names = [p['name'] for p in label_map if p['name'] not in args.discard_labels ]
label_map = dict(zip(range(len(names)),names))
label_map

In [ ]:
yolo_config.update({'names':label_map,'nc':len(label_map)})
yolo_config

In [ ]:
with open(r"D:\PhD\Data per camp\IdentificationDataset\data_config.yaml",'w') as file:
    yaml.dump(yolo_config,file,default_flow_style=False, sort_keys=False)

# Dataset distribution

In [ ]:
import yaml
import pandas as pd
import os
from pathlib import Path

In [ ]:
# load yaml
with open(r"D:\PhD\Data per camp\Extra training data\WAID\data_config.yaml",'r') as file:
    yolo_config = yaml.load(file,Loader=yaml.FullLoader)
yolo_config

In [ ]:
label_map = yolo_config['names']

In [ ]:
split = 'train'

path_dataset = os.path.join(yolo_config['path'],yolo_config[split][0])
path_dataset = path_dataset.replace('images','labels')

path_dataset

In [ ]:
labels = list()

for txtfile in Path(path_dataset).glob("*.txt"):

    df = pd.read_csv(txtfile,sep=" ",names = ['class','x','y','w','h'] )
    df['class'] = df['class'].astype(int)    
    df['image'] = txtfile.stem
    labels.append(df)


In [ ]:
df = pd.concat(labels,axis=0)
df['class'] = df['class'].map(label_map)

In [ ]:
images_per_class = dict()
for cls in df['class'].unique():
    num_imge = df.loc[df['class'] == cls,'image'].unique().shape[0]
    images_per_class[cls] = num_imge

In [ ]:
print("Split:", split)
print(images_per_class)

In [ ]:
print('Split:',split)
print(df['class'].value_counts())

In [ ]:
df['class'].value_counts().plot(kind='bar',figsize=(10,5),logy=True,title=f"{split} label distribution")

# Computing metrics on Validation set

In [40]:
from ultralytics import YOLO
# from pathlib import Path
import torch

In [35]:
# Load a model
path = r"C:\Users\fadel\OneDrive\Bureau\WILD-AI\datalabeling\base_models_weights\yolov8-wildai-obb.pt"
# path = r"C:\Users\fadel\OneDrive\Bureau\WILD-AI\datalabeling\base_models_weights\yolov5su.pt"
model = YOLO(path)  

In [36]:
pred = model.predict(r"C:\Users\fadel\OneDrive\Bureau\WILD-AI\datalabeling\data\train_wildai\images\01f1653a94f14044bf11d78c5b4221d1.JPG")


image 1/1 C:\Users\fadel\OneDrive\Bureau\WILD-AI\datalabeling\data\train_wildai\images\01f1653a94f14044bf11d78c5b4221d1.JPG: 480x640 1815.5ms
Speed: 17.0ms preprocess, 1815.5ms inference, 9.4ms postprocess per image at shape (1, 3, 480, 640)


In [38]:
[result.obb for result in pred]

[ultralytics.engine.results.OBB object with attributes:
 
 cls: tensor([0., 0., 0., 0.])
 conf: tensor([0.6630, 0.3514, 0.2796, 0.2695])
 data: tensor([[6.8862e+02, 1.2766e+03, 8.7202e+01, 8.0733e+01, 1.4624e-01, 6.6303e-01, 0.0000e+00],
         [2.7321e+02, 2.2809e+03, 7.7099e+01, 6.9344e+01, 1.9605e+00, 3.5140e-01, 0.0000e+00],
         [6.2843e+02, 1.4832e+03, 7.9159e+01, 5.8002e+01, 4.8534e-01, 2.7962e-01, 0.0000e+00],
         [3.5846e+02, 1.2539e+03, 7.7243e+01, 6.7475e+01, 5.9167e-01, 2.6951e-01, 0.0000e+00]])
 id: None
 is_track: False
 orig_shape: (3000, 4000)
 shape: torch.Size([4, 7])
 xywhr: tensor([[6.8862e+02, 1.2766e+03, 8.7202e+01, 8.0733e+01, 1.4624e-01],
         [2.7321e+02, 2.2809e+03, 7.7099e+01, 6.9344e+01, 1.9605e+00],
         [6.2843e+02, 1.4832e+03, 7.9159e+01, 5.8002e+01, 4.8534e-01],
         [3.5846e+02, 1.2539e+03, 7.7243e+01, 6.7475e+01, 5.9167e-01]])
 xyxy: tensor([[ 639.5987, 1230.2849,  737.6347, 1322.8635],
         [ 226.4875, 2232.0591,  319.9226, 

In [47]:
pred[0].obb.xyxy

tensor([[ 639.5987, 1230.2849,  737.6347, 1322.8635],
        [ 226.4875, 2232.0591,  319.9226, 2329.7217],
        [ 579.8905, 1439.1091,  676.9661, 1527.3413],
        [ 307.5852, 1204.3851,  409.3313, 1303.4718]])

In [48]:
pred[0].obb.cls

tensor([0., 0., 0., 0.])

In [49]:
pred[0].obb.conf

tensor([0.6630, 0.3514, 0.2796, 0.2695])

In [ ]:
# Customize validation settings
validation_results = model.val(data=r"C:\Users\fadel\OneDrive\Bureau\WILD-AI\datalabeling\data\data_config.yaml",
                                imgsz=640,
                                batch=8,
                                conf=0.25,
                                iou=0.5,
                                device="cpu")

In [ ]:
# Compute predictions
from datalabeling.annotator import Detector

handler = Detector(path_to_weights=path,confidence_threshold=0.3)
predictions = handler.predict_directory(r"C:\Users\fadel\OneDrive\Bureau\WILD-AI\datalabeling\data\train_wildai\images")